In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import streamlit as st

In [4]:
ok = pd.read_csv("100_tick.csv")
symbols = ok['Symbol']

In [5]:
symbols

0            ACC.NS
1       ADANIENT.NS
2     ADANIGREEN.NS
3     ADANIPORTS.NS
4           ATGL.NS
          ...      
95    ULTRACEMCO.NS
96    MCDOWELL-N.NS
97          VEDL.NS
98         WIPRO.NS
99        ZOMATO.NS
Name: Symbol, Length: 100, dtype: object

In [6]:
import ta
import yfinance as yf
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

# Download Apple stock data
symbol = 'ACC.NS'
data = yf.download(symbol, period='MAX')
data['Close_next'] = data['Close'].shift(-1)
data['Close_pct_change'] = data['Close_next'].pct_change()
data = data.fillna(0)


[*********************100%***********************]  1 of 1 completed


In [7]:
data

,Open,High,Low,Close,Adj Close,Volume,Close_next,Close_pct_change
Date,,,,,,,,
2002-07-01,160.000000,163.000000,159.000000,161.750000,108.968071,659631,160.399994,0.000000
2002-07-02,161.050003,163.350006,160.000000,160.399994,108.058571,282660,159.800003,-0.003741
2002-07-03,161.000000,162.000000,158.850006,159.800003,107.654396,500553,158.000000,-0.011264
2002-07-04,159.149994,160.899994,157.500000,158.000000,106.441750,284181,157.550003,-0.002848
2002-07-05,158.050003,159.750000,157.100006,157.550003,106.138626,252587,163.550003,0.038083
...,...,...,...,...,...,...,...,...
2023-03-29,1592.349976,1649.000000,1592.349976,1625.900024,1625.900024,1012619,1667.199951,0.025401
2023-03-31,1640.000000,1672.000000,1630.000000,1667.199951,1667.199951,638264,1709.000000,0.025072
2023-04-03,1675.000000,1714.000000,1651.599976,1709.000000,1709.000000,547370,1687.949951,-0.012317


In [8]:

# Calculate technical indicators
data = pd.DataFrame(ta.add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True))




c:\Users\yasht\anaconda3\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\yasht\anaconda3\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [9]:
data

,Open,High,Low,Close,Adj Close,Volume,Close_next,Close_pct_change,volume_adi,volume_obv,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2002-07-01,160.000000,163.000000,159.000000,161.750000,108.968071,659631,160.399994,0.000000,2.473616e+05,659631,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,161.750000,-85.801637,0.000000,0.000000
2002-07-02,161.050003,163.350006,160.000000,160.399994,108.058571,282660,159.800003,-0.003741,3.220137e+04,376971,...,-0.066621,-0.013324,-0.053297,-4.760395,-0.952079,-3.808316,161.131485,-0.834625,-0.838128,-0.834625
2002-07-03,161.000000,162.000000,158.850006,159.800003,107.654396,500553,158.000000,-0.011264,-1.664312e+05,-123582,...,-0.147792,-0.040218,-0.107574,-5.773029,-1.916269,-3.856760,160.547369,-0.374059,-0.374760,-1.205562
2002-07-04,159.149994,160.899994,157.500000,158.000000,106.441750,284181,157.550003,-0.002848,-3.670294e+05,-407763,...,-0.299043,-0.091983,-0.207061,-9.603748,-3.453765,-6.149983,159.450525,-1.126410,-1.132802,-2.318393
2002-07-05,158.050003,159.750000,157.100006,157.550003,106.138626,252587,163.550003,0.038083,-5.338325e+05,-660350,...,-0.436955,-0.160977,-0.275978,-13.296356,-5.422283,-7.874073,158.699354,-0.284808,-0.285215,-2.596598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29,1592.349976,1649.000000,1592.349976,1625.900024,1625.900024,1012619,1667.199951,0.025401,-5.026743e+06,443446108,...,-3.690574,-3.397778,-0.292797,-14.835722,-21.097059,6.261337,1710.175644,0.740424,0.737696,905.193214
2023-03-31,1640.000000,1672.000000,1630.000000,1667.199951,1667.199951,638264,1709.000000,0.025072,-4.534369e+06,444084372,...,-3.600539,-3.438330,-0.162209,-14.312284,-19.740104,5.427820,1707.566270,2.540127,2.508402,930.726399
2023-04-03,1675.000000,1714.000000,1651.599976,1709.000000,1709.000000,547370,1687.949951,-0.012317,-4.074719e+06,444631742,...,-3.294637,-3.409591,0.114954,-14.825954,-18.757274,3.931319,1707.582963,2.507201,2.476286,956.568779


In [12]:

data['Close_pct_change'] = data['Close_pct_change'] * 100

index = pd.DataFrame(data.columns)
index = index[0].to_numpy()[8:]


In [13]:
index

array(['volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em',
       'volume_sma_em', 'volume_vpt', 'volume_vwap', 'volume_mfi',
       'volume_nvi', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch',
       'volatility_kcl', 'volatility_kcw', 'volatility_kcp',
       'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
       'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg',
       'trend_vortex_ind_diff', 'trend_trix', 'trend_mass_index',
       'trend_dpo', 'trend_kst', 'trend_kst_sig', 'trend_kst_diff',
       'trend_ichimoku_conv', 'trend_ichimoku_base', 'trend_ichimoku_a',
       'tren

In [ ]:

index = index.tolist()
X = data[index].values
y = data[['Close_next', 'Close_pct_change']].values

# Train the KNN model
model = KNeighborsRegressor()
model.fit(X, y)

# Prepare feature data for the next day
last_day_data = data.iloc[-1][index].values.reshape(1, -1)  # Use the last day's technical indicator values

# Predict the next closing price and percentage change
next_close, next_pct_change = model.predict(last_day_data)[0]

# Print the predicted results
if next_pct_change > 2.0:
    print(symbol)
    print(f'Predicted next closing price: {next_close:.2f}')
    print(f'Predicted next percentage change: {next_pct_change:.2f}%')


In [135]:
for i in symbols:
    predict(i)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [155]:
import ta
import yfinance as yf
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import requests
from datetime import date, timedelta
from IPython.display import clear_output

class Prediction:
    
    @staticmethod
    def send_message(bot_token, bot_chatID, message):
        url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
        data = {"chat_id": bot_chatID, "text": message, "parse_mode": "Markdown"}
        response = requests.post(url, data=data)
        return response.json()

    @staticmethod
    def predict_next_close(sym):
        try:
            data = yf.download(sym, period='3y')
            
            data['Close_next'] = data['Close'].shift(-1)
            data['Close_pct_change'] = data['Close_next'].pct_change()
            data = data.fillna(0)

            # Calculate technical indicators
            data = pd.DataFrame(ta.add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True))


            data['Close_pct_change'] = data['Close_pct_change'] * 100

            index = pd.DataFrame(data.columns)
            index = index[0].to_numpy()[:-2]
            index = index.tolist()
            X = data[index].values
            y = data[['Close_next', 'Close_pct_change']].values

            # Train the KNN model
            model = KNeighborsRegressor()
            model.fit(X, y)

            # Prepare feature data for the next day
            last_day_data = data.iloc[-1][index].values.reshape(1, -1)  # Use the last day's technical indicator values

            # Predict the next closing price and percentage change
            next_close, next_pct_change = model.predict(last_day_data)[0]

            return next_close, next_pct_change

        except Exception as e:
            print(f"Error occurred: {e}")
            return None


if __name__ == '__main__':
    
    today = date.today()
    two_days_later = today + timedelta(days=1)
    message = f"predictions for {two_days_later} using Day timeframe"
    Prediction.send_message(bot_token, bot_chatID, message)
    ok = pd.read_csv("100_tick.csv")
    symbols = ok['Symbol']
    bot_token = '6111156932:AAGWg7uRvD7cwTK_UpzGcWv54MJkKWgyMoI'
    bot_chatID = '5562607566'
    for sym in symbols:
        prediction = Prediction.predict_next_close(sym)
        if prediction and abs(prediction[1]) > 1.0:
            message = f"{sym}\nclosing: {prediction[0]:.2f}\nchange: {prediction[1]:.2f}%"
            Prediction.send_message(bot_token, bot_chatID, message)
        clear_output()
